# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2018/04	Concepts, approaches, and avenues for modelling crop health and crop losses	European Journal of Agronomy	This article addresses the modelling of crop health and its impact on crop losses, with a special emphasis on plant diseases. 	Savary, S., Nelson, A.D., Djurle, A., Esker, P.D., Sparks, A., Amorim, L., Bergamin Filho, A., Caffi, T., Castilla, N., Garrett, K. and McRoberts, N., 2018. Concepts, approaches, and avenues for modelling crop health and crop losses. European Journal of Agronomy.	S_Savary_2018	https://www.sciencedirect.com/science/article/pii/S1161030118300832
2017/06/23	Spatial and Temporal Patterns of Rice Production	The Future Rice Strategy for India	This chapter analyzes the spatial and temporal patterns of rice production and productivity in India. Of the 20 agroecological zones, five account for more than 60% of the total rice area and these zones unfortunately house a large number of low-productivity dis

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2018/04,"Concepts, approaches, and avenues for modellin...",European Journal of Agronomy,This article addresses the modelling of crop h...,"Savary, S., Nelson, A.D., Djurle, A., Esker, P...",S_Savary_2018,https://www.sciencedirect.com/science/article/...
1,2017/06/23,Spatial and Temporal Patterns of Rice Production,The Future Rice Strategy for India,This chapter analyzes the spatial and temporal...,Chap. Spatial and Temporal Patterns of Rice Pr...,E_Kannan_2017,http://www.sciencedirect.com/science/article/p...
2,2017/09,bomrang: Fetch Australian Government Bureau of...,The Journal of Open Source Software,The Australian Bureau of Meteorology (BoM) pub...,"Sparks AH, Padgham M, Parsonage H and Pembleto...",sparks_et_al_2017a,http://joss.theoj.org/papers/350bf005bded599e4...
3,2017/07/19,Mungbean and sorghum disease update,GRDC Updates,NaN,"Mungbean and Sorghum Disease Update, L Kelly, ...",kelly_et_al_2017,https://grdc.com.au/resources-and-publications...
4,2015/12/19,Spatial modelling of rice yield losses in Tanz...,Climatic Change,Rice is the most rapidly growing staple food i...,Spatial modelling of rice yield losses in Tanz...,duku_et_al_2016,http://dx.doi.org/10.1007/s10584-015-1580-2
5,2011/08/18,A metamodeling framework for extending the app...,Ecosphere,Process-based ecological models used to assess...,A metamodeling framework for extending the app...,sparks_et_al_2011,Sparks_et_al-2011-Ecosphere.pdf
6,2013/03/10,"A review on crop losses, epidemiology and dise...",European Journal of Plant Pathology,Rice brown spot (BS) is a chronic disease that...,"A review on crop losses, epidemiology and dise...",barnwal_et_al_2013,art%3A10.1007%2Fs10658-013-0195-6.pdf
7,2017/03/17,Crop health and its global impacts on the comp...,Food Security,The literature on the importance of plant path...,Crop health and its global impacts on the comp...,savary_et_al_2017,http://dx.doi.org/10.1007/s12571-017-0659-1
8,2015/05/05,Decision tools for bacterial blight resistance...,Frontiers in Plant Science,Attempting to achieve long-lasting and stable ...,Decision tools for bacterial blight resistance...,dosa_et_al_2015,fpls-06-00305.pdf
9,2014/05/30,Climate change may have limited effect on glob...,Global Change Biology,Weather affects the severity of many plant dis...,Climate change may have limited effect on glob...,sparks_et_al_2014,10.1111/gcb.12587


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

01-cheatham_et_al_2009.md 17-savary_et_al_2017.md   31-esker_et_al_2007a.md
01-savary_et_al_2011.md   18-sparks_et_al_2011.md   31-esker_et_al_2007b.md
04-S_Savary_2018.md       19-duku_et_al_2016.md     31-garrett_et_al_2007.md
05-dosa_et_al_2015.md     19-kelly_et_al_2017.md    31-mckinley_et_al_2012.md
09-sparks_et_al_2017a.md  23-E_Kannan_2017.md       31-sparks_2017.md
10-barnwal_et_al_2013.md  28-laborte_et_al_2015.md  31-sparks_et_al_2008a.md
10-garrett_et_al_2011.md  28-sparks_et_al_2017.md   31-sparks_et_al_2008b.md
12-gaudin_et_al_2012.md   30-sparks_et_al_2014.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
